In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

In [ ]:
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Flatten, Conv1D, MaxPooling1D, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import HeNormal
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import BatchNormalization
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import SimpleRNN, Dropout, Dense

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, cohen_kappa_score
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPool1D, Flatten,LSTM

In [ ]:
train = pd.read_csv('/content/LSA+Word2vec_Tr.csv')
test = pd.read_csv('/content/LSA+Word2vec_Ind.csv')

In [ ]:
xtrain = train.drop(['Target'], axis = 1)
ytrain = train['Target']

In [ ]:
xtest = test.drop(['Target'], axis = 1)
ytest = test['Target']

In [ ]:
X = xtrain
Y = ytrain

In [ ]:
X = X.to_numpy()
X = X.reshape(X.shape[0], X.shape[1], 1)

In [ ]:
kf = KFold(n_splits=5, shuffle=True)
for train_index, val_index in kf.split(X):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = Y[train_index], Y[val_index]

In [ ]:
    # Build CNN + RNN model
    model = Sequential()

    # CNN layers
    model.add(Conv1D(filters=256, kernel_size=3, activation='relu', input_shape=(xtrain.shape[1], 1)))
    model.add(Conv1D(filters=176, kernel_size=2, activation='relu'))
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(BatchNormalization())

    # RNN layers
    model.add(SimpleRNN(256, activation='tanh', return_sequences=True))
    model.add(BatchNormalization())
    model.add(SimpleRNN(128, activation='tanh', return_sequences=False))
    model.add(BatchNormalization())

    # Fully connected layers
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Binary classification

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

    # Train the model
    history = model.fit(X_train, y_train, epochs=75, batch_size=64)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/75
14/14 ━━━━━━━━━━━━━━━━━━━━ 19s 398ms/step - accuracy: 0.7823 - loss: 0.3825
Epoch 2/75
14/14 ━━━━━━━━━━━━━━━━━━━━ 5s 337ms/step - accuracy: 1.0000 - loss: 0.0213
Epoch 3/75
14/14 ━━━━━━━━━━━━━━━━━━━━ 7s 488ms/step - accuracy: 1.0000 - loss: 0.0124
Epoch 4/75
14/14 ━━━━━━━━━━━━━━━━━━━━ 8s 344ms/step - accuracy: 1.0000 - loss: 0.0073
Epoch 5/75
14/14 ━━━━━━━━━━━━━━━━━━━━ 7s 465ms/step - accuracy: 1.0000 - loss: 0.0056
Epoch 6/75
14/14 ━━━━━━━━━━━━━━━━━━━━ 5s 339ms/step - accuracy: 1.0000 - loss: 0.0044
Epoch 7/75
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 436ms/step - accuracy: 1.0000 - loss: 0.0034
Epoch 8/75
14/14 ━━━━━━━━━━━━━━━━━━━━ 5s 364ms/step - accuracy: 1.0000 - loss: 0.0030
Epoch 9/75
14/14 ━━━━━━━━━━━━━━━━━━━━ 12s 487ms/step - accuracy: 1.0000 - loss: 0.0032
Epoch 10/75
14/14 ━━━━━━━━━━━━━━━━━━━━ 8s 335ms/step - accuracy: 1.0000 - loss: 0.0031
Epoch 11/75
14/14 ━━━━━━━━━━━━━━━━━━━━ 7s 489ms/step - accuracy: 1.0000 - loss: 0.0024
Epoch 12/75
14/14 ━━━━━━━━━━━━━━━━━━━━ 8s 349ms/st

In [ ]:
pred = model.predict(X_val)
y_pred_classes = np.round(pred).astype(int)

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 116ms/step


In [ ]:
accuracy_score(y_val, y_pred_classes), recall_score(y_val, y_pred_classes), precision_score(y_val, y_pred_classes), cohen_kappa_score(y_val, y_pred_classes), matthews_corrcoef(y_val, y_pred_classes)

(1.0, 1.0, 1.0, 1.0, 1.0)

In [ ]:
f1_score(y_val, y_pred_classes)

1.0

In [ ]:
cm1 = confusion_matrix(y_val, y_pred_classes)
specificity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
specificity

1.0

Test

In [ ]:
sample_size = xtrain.shape[0] # number of samples in train set
time_steps  = xtrain.shape[1] # number of features in train set
input_dimension = 1               # each feature is represented by 1 number

In [ ]:
train_data_reshaped = xtrain.values.reshape(sample_size,time_steps,input_dimension)
n_timesteps = train_data_reshaped.shape[1]
n_features  = train_data_reshaped.shape[2]

In [ ]:
# Build CNN + RNN model
model1 = Sequential()

# CNN layers
model1.add(Conv1D(filters=256, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
model1.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
model1.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model1.add(MaxPooling1D(pool_size=2))
model1.add(BatchNormalization())

# RNN layers
model1.add(SimpleRNN(128, activation='tanh', return_sequences=True))
model1.add(SimpleRNN(64, activation='tanh', return_sequences=False))

# Fully connected layers
model1.add(Dense(64, activation='relu'))
model1.add(Dense(1, activation='sigmoid'))  # Binary classification

# Compile the model
model1.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model1.fit(xtrain, ytrain, epochs = 75, batch_size= 32)

Epoch 1/75
34/34 ━━━━━━━━━━━━━━━━━━━━ 13s 227ms/step - accuracy: 0.9051 - loss: 0.3563
Epoch 2/75
34/34 ━━━━━━━━━━━━━━━━━━━━ 9s 258ms/step - accuracy: 1.0000 - loss: 0.0503
Epoch 3/75
34/34 ━━━━━━━━━━━━━━━━━━━━ 7s 202ms/step - accuracy: 1.0000 - loss: 0.0214
Epoch 4/75
34/34 ━━━━━━━━━━━━━━━━━━━━ 12s 245ms/step - accuracy: 1.0000 - loss: 0.0127
Epoch 5/75
34/34 ━━━━━━━━━━━━━━━━━━━━ 11s 263ms/step - accuracy: 1.0000 - loss: 0.0085
Epoch 6/75
34/34 ━━━━━━━━━━━━━━━━━━━━ 8s 206ms/step - accuracy: 1.0000 - loss: 0.0061
Epoch 7/75
34/34 ━━━━━━━━━━━━━━━━━━━━ 9s 265ms/step - accuracy: 1.0000 - loss: 0.0047
Epoch 8/75
34/34 ━━━━━━━━━━━━━━━━━━━━ 9s 232ms/step - accuracy: 1.0000 - loss: 0.0037
Epoch 9/75
34/34 ━━━━━━━━━━━━━━━━━━━━ 8s 217ms/step - accuracy: 1.0000 - loss: 0.0031
Epoch 10/75
34/34 ━━━━━━━━━━━━━━━━━━━━ 11s 258ms/step - accuracy: 1.0000 - loss: 0.0025
Epoch 11/75
34/34 ━━━━━━━━━━━━━━━━━━━━ 9s 229ms/step - accuracy: 1.0000 - loss: 0.0021
Epoch 12/75
34/34 ━━━━━━━━━━━━━━━━━━━━ 8s 224ms/

In [ ]:
pred1 = model1.predict(xtest)
pred1 = (pred1 > 0.5)

9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 99ms/step


In [ ]:
accuracy_score(ytest, pred1), precision_score(ytest, pred1), recall_score(ytest, pred1), f1_score(ytest, pred1), cohen_kappa_score(ytest, pred1), matthews_corrcoef(ytest, pred1)

(0.9962406015037594,
 0.9925373134328358,
 1.0,
 0.9962546816479401,
 0.9924812030075187,
 0.9925092578236596)

In [ ]:
cm1 = confusion_matrix(ytest, pred1)
specificity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
specificity

0.9924812030075187